Installation

In [19]:
pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 opencv-python matplotlib

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Import Dependencies

In [20]:
import tensorflow as tf 
import numpy as np
from matplotlib import pyplot as plt 
import cv2

Load Model

In [21]:
interpreter2 = tf.lite.Interpreter(model_path='lite-model_movenet_singlepose_thunder_3.tflite')
interpreter2.allocate_tensors()

In [22]:
interpreter2.get_input_details()

[{'name': 'serving_default_input:0',
  'index': 0,
  'shape': array([  1, 256, 256,   3]),
  'shape_signature': array([  1, 256, 256,   3]),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [23]:
interpreter2.get_output_details()

[{'name': 'StatefulPartitionedCall:0',
  'index': 312,
  'shape': array([ 1,  1, 17,  3]),
  'shape_signature': array([ 1,  1, 17,  3]),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

Draw Keypoints

In [24]:
# Here confidence_threshold is the threshhold above which we actually want to draw our keypoints
def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y, x, 1]))
    # In the above codeline, we are taking keypoints and multiplying them with the frame shape using np.multiply
    # And np.squeeze is for the easy access of the array

    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 6, (0, 255, 0), -1)
            # cv2.circle(frame, x and y coordinates wrapping inside int, how big our circle would be, color of the circle, how thick the circle line(if we put it -1, it fills the circle with the given color))
            

Draw Edges

In [25]:
Edges = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

In [26]:
def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape 
    shaped = np.squeeze(np.multiply(keypoints, [y, x, 1]))

    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]

        if (c1 > confidence_threshold) & (c2 > confidence_threshold):
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 2)
            # cv2.line(frame, first point, second point, color, line width)
    

Make Pose Detections

In [27]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()

    # Reshape image
    img = frame.copy()
    img = tf.image.resize_with_pad(np.expand_dims(img, axis = 0), 256, 256)
    input_image = tf.cast(img, dtype = tf.float32)

    # Setup input and output
    input_details = interpreter2.get_input_details()
    output_details = interpreter2.get_output_details()

    # Make predictions
    interpreter2.set_tensor(input_details[0]['index'], np.array(input_image))
    interpreter2.invoke()
    keypoints_with_scores = interpreter2.get_tensor(output_details[0]['index'])
    print(keypoints_with_scores)

    # Rendering
    draw_connections(frame, keypoints_with_scores, Edges, 0.4)
    draw_keypoints(frame, keypoints_with_scores, 0.4)

    cv2.imshow('MoveNet Lightning', frame)
    
    if cv2.waitKey(10) & 0xFF==ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

[[[[0.78743947 0.5912666  0.48655728]
   [0.7246957  0.63865894 0.68509865]
   [0.73389083 0.5238456  0.5006413 ]
   [0.7728535  0.7116294  0.49911532]
   [0.7815351  0.42523527 0.5085433 ]
   [0.85394686 0.8514242  0.12755099]
   [0.87697357 0.40724832 0.03060281]
   [0.82176745 0.8993143  0.01165959]
   [0.40967125 0.0223255  0.02523735]
   [0.64920956 0.8026255  0.02396384]
   [0.70931345 0.4468447  0.01909715]
   [0.822845   0.88695836 0.03896126]
   [0.82143265 0.852975   0.04828125]
   [0.8468449  0.86196446 0.0211862 ]
   [0.82093626 0.3553275  0.00398964]
   [0.70489496 0.6366651  0.030341  ]
   [0.70267385 0.44097328 0.02035543]]]]
[[[[0.7854428  0.6050952  0.5057956 ]
   [0.72416484 0.6513031  0.7184628 ]
   [0.73076123 0.53653127 0.53498054]
   [0.77270555 0.7217587  0.39229202]
   [0.7742653  0.4325514  0.32961857]
   [0.85118675 0.85204995 0.17570803]
   [0.8703654  0.42083773 0.04165617]
   [0.80598295 0.8987668  0.0199185 ]
   [0.40997142 0.02153775 0.0410631 ]
   [0.798

In [28]:
keypoints_with_scores[0][0]

"""
These shows 17 keypoints of our body, in the following order:
(1) Nose
(2) Left Eye
(3) Right Eye
(4) Left Ear
(5) Right Ear
(6) Left Shoulder
(7) Right Shoulder
(8) Left Elbow
(9) Right Elbow
(10) Left Wrist
(11) Right Wrist
(12) Left Hip
(13) Right Hip
(14) Left Knee
(15) Right Knee
(16) Left Ankle
(17) Right Ankle
"""

'\nThese shows 17 keypoints of our body, in the following order:\n(1) Nose\n(2) Left Eye\n(3) Right Eye\n(4) Left Ear\n(5) Right Ear\n(6) Left Shoulder\n(7) Right Shoulder\n(8) Left Elbow\n(9) Right Elbow\n(10) Left Wrist\n(11) Right Wrist\n(12) Left Hip\n(13) Right Hip\n(14) Left Knee\n(15) Right Knee\n(16) Left Ankle\n(17) Right Ankle\n'